In [120]:
### getting the libraries
import pandas as pd
#!pip install -U imbalanced-learn   
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
### check the distribution of all the columns
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from category_encoders import TargetEncoder
import xgboost as xgb
from sklearn import metrics

from sklearn.metrics import accuracy_score
### reading  the training csv data
train=pd.read_csv('train (6).csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 230 entries, Var1 to Var230
dtypes: float64(191), int64(1), object(38)
memory usage: 87.7+ MB


In [129]:
null_counts = train.isnull().sum()
null_counts

Var1      49298
Var2      48759
Var3      48760
Var4      48421
Var5      48513
          ...  
Var226        0
Var227        0
Var228        0
Var229    28432
Var230    50000
Length: 230, dtype: int64

In [ ]:
### missing data imputation
threshold = len(train) * 0.2
train_cleaned=train.dropna(axis=1, thresh=threshold)
train_cleaned.interpolate(method ='linear', limit_direction ='forward', inplace=True)
train_cleaned

In [ ]:
### check numerical vs categorical columns
train_cleaned.dtypes

In [ ]:
### add the labes from other csv
labels=pd.read_csv('train_churn_labels.csv')
train_cleaned['churn']=labels
train_cleaned['churn']=train_cleaned['churn'].apply(lambda x : 0 if x==-1 else 1 )

In [ ]:
### scaling data and checking the descriptives
scaler = StandardScaler()
numerical_df = train_cleaned.select_dtypes(include=[np.int64, np.float64])
categorical_df=train_cleaned.select_dtypes(exclude=[np.int64, np.float64])
X_train_scaled = scaler.fit_transform(numerical_df)
df2 = pd.DataFrame(X_train_scaled, columns=numerical_df.columns)
#X_test_scaled = scaler.transform(X_test)
### summary statistics for all the columns
df2.describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

In [ ]:
columns=list(range(0,50))

In [ ]:
#### shows how what are the columsn with more variance or data
train_cleaned.nunique().sort_values(ascending=False)[:50]

In [ ]:
### encoding the variables based on probability of classes in the target column
encoder = TargetEncoder()
for i in categorical_df.columns:
   train_cleaned[i+'Encoded'] = encoder.fit_transform(train_cleaned[i], train_cleaned['churn'])


In [ ]:
### histogram splot for distributions of the column
x=pd.DataFrame(df2, columns=train_cleaned.columns)
fig, ax = plt.subplots(ncols=6, nrows=5, figsize=(20, 20))
index = 0
ax = ax.flatten()

for col in x.columns:
    if index<30:
        sns.histplot(x[col], ax=ax[index])
    index+=1
plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

In [ ]:
### box plots
fig , ax = plt.subplots(3,2,figsize=(7,7))
sns.boxplot(x=df2['Var6'],ax=ax[0,0])
sns.boxplot(x=df2['Var7'],ax=ax[0,1])
sns.boxplot(x=df2['Var13'],ax=ax[1,0])
sns.boxplot(x=df2['Var21'],ax=ax[1,1])
sns.boxplot(x=df2['Var22'],ax=ax[2,0])
sns.boxplot(x=df2['Var24'],ax=ax[2,1])


In [ ]:
# Visualize tumor characteristics for positive and negatives classes 


# Instantiate figure object
fig = plt.figure()


Nchurn = train_cleaned[train_cleaned['churn']==-1]
Churn = train_cleaned[train_cleaned['churn']==1]

for i,b in enumerate(list(train_cleaned.columns[0:20])):
    
    # Enumerate starts at index 0, need to add 1 for subplotting
    i +=1
    
    # Create axes object for position i
    ax = fig.add_subplot(3,4,i)
    
    # Plot via histogram tumor charateristics using stacked and alpha parameters for..
    # comparisons.
  
    ax.hist(Nchurn[b], label = 'no-churn', stacked = True, alpha=0.5, color= 'b')
    ax.hist(Churn[b], label= 'churn', stacked = True, alpha=0.5, color= 'r')
    
    ax.set_title(b)

sns.set_style("whitegrid")
plt.tight_layout()
plt.legend()
plt.show()   

In [ ]:
### distirbution plots for some columns
fig , ax = plt.subplots(3,2,figsize=(7,7))
sns.distplot(x=df2['Var7'],ax=ax[0,0])
sns.distplot(x=df2['Var6'],ax=ax[0,1])
sns.distplot(x=df2['Var13'],ax=ax[1,0])
sns.distplot(x=df2['Var21'],ax=ax[1,1])
sns.distplot(x=df2['Var22'],ax=ax[2,0])
sns.distplot(x=df2['Var24'],ax=ax[2,1])

In [ ]:
### scatter plots
sns.set(rc={'figure.figsize':(20,30)})
cData_attr = df2.iloc[:, 5:20]
sns.pairplot(cData_attr, diag_kind='kde') 

In [ ]:
### correlogram
plt.figure(figsize=(16,9))
sns.heatmap(train_cleaned.iloc[:, 20:50].corr(numeric_only=True),annot=True)

In [ ]:
### outlier capping  for all values
for col in numerical_df.columns:
    percentiles = numerical_df[col].quantile([0.01, 0.99]).values
    numerical_df[col] = np.clip(numerical_df[col], percentiles[0], percentiles[1])

In [ ]:
### Distribution of the target classes 
labels['Label'].value_counts()

In [ ]:

### Distribution of the target classes 
### 0 is malignant and 1 is benign
import seaborn as sns
print('no: ', len(train_cleaned[train_cleaned['churn']==-1])/len(train_cleaned)*100, '%')
print('Yes: ', len(train_cleaned[train_cleaned['churn']==1])/len(train_cleaned)*100, '%')
sns.countplot(train_cleaned,x='churn', legend=True, hue='churn')
plt.xlabel("target")
plt.title("no-churn vs churn points")

In [123]:
### preparing fo rthe modelling
numerical_df = train_cleaned.select_dtypes(include=[np.int64, np.float64])
# Label / data separation
numerical_df.fillna(0,inplace=True)
X = numerical_df.drop('churn', axis=1)
y = numerical_df['churn']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [113]:
# Training the model
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.954

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      9305
           1       0.74      0.52      0.61       695

    accuracy                           0.95     10000
   macro avg       0.85      0.75      0.79     10000
weighted avg       0.95      0.95      0.95     10000


Confusion Matrix:
[[9181  124]
 [ 336  359]]


In [95]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': abs(model.coef_[0])
}).sort_values('importance', ascending=False)

print("\nTop 10 Important Features:")
print(feature_importance.head(10))


Top 10 Important Features:
          feature  importance
71  Var217Encoded    1.233422
63  Var200Encoded    0.788366
70  Var214Encoded    0.788366
62  Var199Encoded    0.701608
72  Var218Encoded    0.536168
59  Var192Encoded    0.340330
52  Var216Encoded    0.306073
28         Var126    0.288667
75  Var225Encoded    0.276150
57  Var227Encoded    0.251276


In [110]:
numerical_df = train_cleaned.select_dtypes(include=[np.int64, np.float64])
# Label / data separation
numerical_df=numerical_df[feature_importance['feature'][0:30]]
X = numerical_df
y = train_cleaned['churn']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [118]:
# Training the model after the feature sleection
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.954

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      9305
           1       0.74      0.52      0.61       695

    accuracy                           0.95     10000
   macro avg       0.85      0.75      0.79     10000
weighted avg       0.95      0.95      0.95     10000


Confusion Matrix:
[[9181  124]
 [ 336  359]]


In [121]:
y_pred_proba = model.predict_proba(X_test)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

0.5135024489811697


c:\Users\sushmita\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [126]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Random Forest Classifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Calculate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
# Calculate Precision
precision = precision_score(y_test, rf_predictions)
# Calculate Recall
recall = recall_score(y_test, rf_predictions)
# Calculate F1-score
f1 = f1_score(y_test, rf_predictions)

# Print the results

print("Accuracy score:",rf_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)

print(f"AUC Score: {auc_score:.2f}")

Accuracy score: 0.9591
Precision: 0.8864864864864865
Recall: 0.4719424460431655
F1-score: 0.615962441314554
AUC Score: 0.96


In [125]:
from sklearn.decomposition import PCA

def apply_pca(X, n_components=0.8):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    print(f"Reduced from {X.shape[1]} to {X_pca.shape[1]} dimensions")
    return X_pca, pca

X_train_pca, pca_model = apply_pca(X_train_scaled)
X_test_pca = pca_model.transform(X_test_scaled)

Reduced from 77 to 33 dimensions


In [100]:
print(y_train.shape)
print(X_train.shape)
print(X_test.shape)
print(y_pred.shape)
print(y_test.shape)

(40000,)
(40000, 77)
(10000, 77)
(10000,)
(10000,)


In [ ]:
len(y_pred)

In [101]:
# Training the model
model = LogisticRegression(random_state=42)
model.fit(X_train_pca, y_train)

# Make predictions
y_pred = model.predict(X_test_pca)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9496

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      9305
           1       0.73      0.44      0.55       695

    accuracy                           0.95     10000
   macro avg       0.84      0.71      0.76     10000
weighted avg       0.94      0.95      0.94     10000


Confusion Matrix:
[[9192  113]
 [ 391  304]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Random Forest Classifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train_pca, y_train)
rf_predictions = rf_model.predict(X_test_pca)

# Calculate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
# Calculate Precision
precision = precision_score(y_test, rf_predictions)
# Calculate Recall
recall = recall_score(y_test, rf_predictions)
# Calculate F1-score
f1 = f1_score(y_test, rf_predictions)

# Print the results


print("Accuracy score:",rf_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy score: 0.9481
Precision: 0.8188405797101449
Recall: 0.3251798561151079
F1-score: 0.4654994850669413


c:\Users\sushmita\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 77 features, but RandomForestClassifier is expecting 33 features as input.

In [127]:


xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Initial Model Accuracy: {accuracy * 100:.2f}%")
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)

print(f"AUC Score: {auc_score:.2f}")

Initial Model Accuracy: 96.06%
AUC Score: 0.97


In [107]:
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      9305
           1       0.77      0.62      0.68       695

    accuracy                           0.96     10000
   macro avg       0.87      0.80      0.83     10000
weighted avg       0.96      0.96      0.96     10000


Confusion Matrix:
[[9178  127]
 [ 267  428]]


In [108]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma': [0, 1, 5],
    'subsample': [0.8, 1.0]
}
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'gamma': [0, 1, 5], 'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7], 'subsample': [0.8, 1.0]},
             scoring='accuracy')

In [109]:
final_accuracy = grid_search.best_score_
print(f"Final Model Accuracy after Tuning: {final_accuracy * 100:.2f}%")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Final Model Accuracy after Tuning: 95.91%
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      9305
           1       0.77      0.62      0.68       695

    accuracy                           0.96     10000
   macro avg       0.87      0.80      0.83     10000
weighted avg       0.96      0.96      0.96     10000


Confusion Matrix:
[[9178  127]
 [ 267  428]]


In [115]:
### testing out on SMOTE methods
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.99      0.98      9305
           1       0.77      0.62      0.68       695

    accuracy                           0.96     10000
   macro avg       0.87      0.80      0.83     10000
weighted avg       0.96      0.96      0.96     10000



In [116]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
model = RandomForestClassifier()
model.fit(X_train_res, y_train_res)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      9305
           1       0.69      0.64      0.66       695

    accuracy                           0.95     10000
   macro avg       0.83      0.81      0.82     10000
weighted avg       0.95      0.95      0.95     10000

